# Tour de Pologne Top 10 finishers prediction part 1 of 3 (data mining)<a class='tocSkip'>
Main idea is to predict who will finish in Top 10 position after each stage.<br>
Whole process is going to be updated and described thouroughly very soon, stay tuned.

**Next notebooks:** <br>
[Part 2: Data stats and visuals](tdp_2_3_data_statistics_visualisation.ipynb)<br>
[Part 3: Data  modeling](tdp_3_3_data_modeling.ipynb)

In [1]:
# %load_ext watermark
%reload_ext watermark
%watermark -a "Norbert Oksza Strzelecki" -v -p numpy,pandas,re,requests,bs4

Norbert Oksza Strzelecki 

CPython 3.6.6
IPython 6.5.0

numpy 1.15.2
pandas 0.23.4
re 2.2.1
requests 2.19.1
bs4 4.6.3


In [1]:
# 07.03.2019
%reload_ext watermark
%watermark -a "Norbert Oksza Strzelecki" -v -p numpy,pandas,re,requests,bs4

Norbert Oksza Strzelecki 

CPython 3.6.7
IPython 6.5.0

numpy 1.16.2
pandas 0.24.1
re 2.2.1
requests 2.19.1
bs4 4.6.3


In [1]:
import numpy as np
import pandas as pd
import time

import re

import requests
from bs4 import BeautifulSoup

In [2]:
# Base URL for web scraping
url_base = 'http://firstcycling.com/race.php?r=19&y='

In [3]:
# Defining year range for data of interest
# Before the year of 2014 there is lack of general classification positions (GC) and time (GC Time) information
tour_year_start = 2014
tour_year_stop = 2018
tour_years = list(range(tour_year_start, tour_year_stop+1))
tour_years

[2014, 2015, 2016, 2017, 2018]

<br><br><br>
# Tour results by stages
<br>

In [4]:
# List of tour years for analysis
url_tour_results = []
for year in tour_years:
    url_tour_results.append(url_base + str(year))
# url_tour_results

In [8]:
# List of all stages in the above chosen tours
url_stages_results = []
for stage in url_tour_results:
    for s in range(1,8):
        url_stages_results.append(stage + '&k=stages&e=0' + str(s))
# url_stages_results

### Generating dictionaries of biker and team name with corresponding system number  that are needed for further steps

In [10]:
biker_dict_stage = {}
team_dict_stage = {}
i = tour_year_start - 1
for stage in [stage for stage in url_stages_results if 'e=01' in stage]:
    # Preparing dataframe with all racer names
    dict_prep_ = pd.read_html(stage)
    dict_prep = dict_prep_[0]
    dict_prep['Name'] = dict_prep['Name'].str[0] + '.' + dict_prep['Name'].str.split(' ', 1, expand=True)[1]
    
    # Scraping system numbers
    r = requests.get(stage)
    soup = BeautifulSoup(r.text,'html.parser')
    system_numbers = soup.find('table')

    # Schema as `rider.php?r=2165` (2-5 digits with margin)
    biker_system_nr = pd.DataFrame(re.findall('rider.php\?r=(\d{2,6})', str(system_numbers)), columns=['biker_sys_nr'])
    # Schema as `team.php?l=8433` (3-4 digits with margin)
    team_system_nr = pd.DataFrame(re.findall('team.php\?l=(\d{3,5})', str(system_numbers)), columns=['team_sys_nr'])
    
    # Merging team and rider names with their system numbers 
    biker_dict_stage_ = dict(zip(dict_prep['Name'], biker_system_nr['biker_sys_nr']))
    biker_dict_stage.update(biker_dict_stage_)
    team_dict_stage_ = dict(zip(dict_prep['Team'], team_system_nr['team_sys_nr']))
    team_dict_stage.update(team_dict_stage_)
    i += 1
    print('Created dictionary of bikers participating in year ' + str(i))

Created dictionary of bikers participating in year 2014
Created dictionary of bikers participating in year 2015
Created dictionary of bikers participating in year 2016
Created dictionary of bikers participating in year 2017
Created dictionary of bikers participating in year 2018


In [11]:
# Overall number of bikers and teams
len(biker_dict_stage), len(team_dict_stage)

(513, 69)

### Defining datetime functions

In [14]:
# Adding leading zeros for further splitting based on ':' character
def zeros_to_sec(df, col1, col2=False):
    df[col1] = np.where(df[col1].str.len() == 2, '00:00:' + df[col1],'00:' + df[col1]) 
    df[col1] = df[col1].where(df[col1].str.len() != 11, df[col1].str.slice(3,11))
    
    if col2:
        df[col2] = np.where(df[col2].str.len() == 2, '00:00:' + df[col2],'00:' + df[col2])
        df[col2] = df[col2].where(df[col2].str.len() != 11, df[col2].str.slice(3,11))

In [15]:
# Changing overall time to seconds
def time_to_sec(df, col1, col2=False):
    df[col1] = [(int(t[0])*3600 + int(t[1])*60 + int(t[2])) for t in df[col1].str.split(':')]
    df[col1] = df[col1] + df[col1].max()
    df.at[0, [col1]] = df[col1].max() / 2

    if col2:
        df[col2] = [(int(t[0])*3600 + int(t[1])*60 + int(t[2])) for t in df[col2].str.split(':')]
        df[col2] = df[col2] + df[col2].max()
        df.at[df[col2].idxmax(), [col2]] = df[col2].max() / 2

### Web scraping tour's stage-by-stage info for adding new columns with extended statistics. Output: `stages`

In [11]:
# Using URLs with all-year stages results
# url_stages_results[:]

In [17]:
stages_ = []

for year in tour_years:
    i = 1
    for stage_ in [stage_ for stage_ in url_stages_results if 'y='+str(year) in stage_]:
        try:
            stages__ = pd.read_html(stage_)
            stages__ = stages__[0]
            stages__['year'] = year
            # Limiting number of stages in each year to 7
            if i <= 7:
                stages__['stage'] = i
            else:
                i = 1        
            stages__.columns = ['position','to_drop','name','age','team','time_in_s','gc','gc_time_in_s','year','stage']
            stages__['age'] = year - stages__['age']
            stages__.replace(to_replace=r'\+ ', value='', regex=True, inplace=True)
            stages__.replace(to_replace=[np.NaN], value='00', inplace=True)
            
            zeros_to_sec(stages__, 'time_in_s','gc_time_in_s')
            time_to_sec(stages__, 'time_in_s','gc_time_in_s')
            
            # Setting DNF and DNS times to some big number
            try:
                stages__['time_in_s'].where((stages__['position'] != 'DNF') & (stages__['position'] != 'DNS'),
                                            999999, inplace=True)
                stages__['gc_time_in_s'].where((stages__['position'] != 'DNF') & (stages__['position'] != 'DNS'),
                                               999999, inplace=True)
            except TypeError:
                pass
            # Setting DNF and DNS poitions to some big number
            stages__.replace(to_replace=['DNF', 'DNS'], value=999, inplace=True)
            stages__['position'] = stages__['position'].astype('int')
            
            stages_.append(stages__)
            print('Scraped tour of', year, '/ Stage', i)
            i += 1
        except TypeError:
            print('There is no data for this stage. Skipping...')
            i += 1
            pass

Scraped tour of 2014 / Stage 1


C:\Users\cyberdeck\Anaconda3\lib\site-packages\pandas\core\ops.py:1649: FutureWarning: elementwise comparison failed; returning scalar instead, but in the future will perform elementwise comparison
  result = method(y)


Scraped tour of 2014 / Stage 2


C:\Users\cyberdeck\Anaconda3\lib\site-packages\pandas\core\ops.py:1649: FutureWarning: elementwise comparison failed; returning scalar instead, but in the future will perform elementwise comparison
  result = method(y)


Scraped tour of 2014 / Stage 3


...

Scraped tour of 2017 / Stage 2
Scraped tour of 2017 / Stage 3
Scraped tour of 2017 / Stage 4
Scraped tour of 2017 / Stage 5
Scraped tour of 2017 / Stage 6
Scraped tour of 2017 / Stage 7


C:\Users\cyberdeck\Anaconda3\lib\site-packages\pandas\core\ops.py:1649: FutureWarning: elementwise comparison failed; returning scalar instead, but in the future will perform elementwise comparison
  result = method(y)


Scraped tour of 2018 / Stage 1


C:\Users\cyberdeck\Anaconda3\lib\site-packages\pandas\core\ops.py:1649: FutureWarning: elementwise comparison failed; returning scalar instead, but in the future will perform elementwise comparison
  result = method(y)


Scraped tour of 2018 / Stage 2
Scraped tour of 2018 / Stage 3
Scraped tour of 2018 / Stage 4
Scraped tour of 2018 / Stage 5
Scraped tour of 2018 / Stage 6
Scraped tour of 2018 / Stage 7


In [21]:
# Merging above stages into DataFrame
stages = pd.concat(stages_, axis=0)
stages.reset_index(drop=True, inplace=True)

# Dropping columns with no information
stages.drop('to_drop', axis=1, inplace=True)

# Shortening biker names into 'initial.surname' form
stages['name'] = stages['name'].str[0] + '.' + stages['name'].str.split(' ', 1, expand=True)[1]

# Creating Biker and Team system numbers
stages['biker_sys_nr'] = stages['name'].map(biker_dict_stage)
stages['team_sys_nr'] = stages['team'].map(team_dict_stage)

# Rearranging columns
stages = stages[['name','biker_sys_nr','age','team','team_sys_nr','time_in_s','gc','gc_time_in_s','year','stage','position']]

In [22]:
stages.head()

,name,biker_sys_nr,age,team,team_sys_nr,time_in_s,gc,gc_time_in_s,year,stage,position
0,Y.Hutarovich,100,31,AG2R La Mondiale,10072,20870.0,1,20860.0,2014,1,1
1,R.Maikin,14004,24,RusVelo,3482,20870.0,2,20864.0,2014,1,2
2,M.Mori,492,34,Lampre - Merida,6418,20870.0,3,20866.0,2014,1,3
3,G.Boivin,1937,25,Cannondale Pro Cycling,3481,20870.0,6,20870.0,2014,1,4
4,M.Haller,6246,23,Team Katusha,8441,20870.0,7,20870.0,2014,1,5


### Features engineering: Columns with different statistics for each biker. Output: `biker_by_stage` 

In [23]:
'''
'n_tours'            - number of tours biker started in
'n_stages'           - number of stages biker started in
'avg_year'           - mean year from all tours started in
'best_pos_ovr'       - best position from all stages
'worst_pos_ovr'      - worst position from all stages
'avg_pos_ovr'        - average position from all stages
'n_top10_ovr'        - number of top10 positions in all stages 
'perc_win_ovr'       - percentage of winning throughout all stages
'perc_top10_ovr'     - percentage of being in top10 in all stages
'avg_speed_ovr'      - average speed throughout all stages
'best_pos_l3'        - best position from last 3 stages
'wors_pos_l3'        - etc...
'avg_pos_l3'
'n_top10_l3'
'perc_win_l3'
'perc_top10_l3'
'avg_speed_l3'
'''

biker_by_stage_ = pd.DataFrame()

# Number of tours and stages ridden by Biker
biker_by_stage_['n_tours'] = stages.groupby(['name'])['year'].nunique()
biker_by_stage_['n_stages'] = stages.groupby(['name'])['stage'].count()
biker_by_stage_['avg_year'] = stages.groupby('name')['year'].mean()

# Best, average and worst tour stats
min_max_ovr = stages.groupby('name')[['position','time_in_s','gc','gc_time_in_s']].agg(['min','mean','max'])

# Number of Top 10 positions
top10 = pd.DataFrame(stages.groupby(stages[stages['position'] <= 10]['name'])['position'].count())

# Number of won stages
winner_ = pd.DataFrame(stages.groupby(stages[stages['position'] == 1]['name'])['position'].count())

# Defining the same data as above but only for last 3 stages
last_3_stages = stages.sort_values(['name','year','stage']).groupby('name').tail(3)
min_max_avg_l3 = last_3_stages.groupby('name')[['position','time_in_s','gc','gc_time_in_s']].agg(['min','mean','max'])
top10_l3 = pd.DataFrame(last_3_stages[last_3_stages['position'] <= 10]['name'].value_counts())
winner_l3 = pd.DataFrame(last_3_stages.groupby(last_3_stages[last_3_stages['position'] == 1]['name'])['position'].count())

# Merging all info into one DataFrame
biker_by_stage_ = pd.concat([biker_by_stage_, min_max_ovr, top10, winner_, min_max_avg_l3, top10_l3, winner_l3], axis=1, sort=False)
biker_by_stage_ = biker_by_stage_.replace(np.NaN, 0)
biker_by_stage_.columns = ['n_tours','n_stages','avg_year','best_pos_ovr','avg_pos_ovr','worst_pos_ovr','best_time_ovr','avg_time_ovr','worst_time_ovr',
                         'best_gc_ovr','avg_gc_ovr','worst_gc_ovr','best_gc_time_ovr','avg_gc_time_ovr','worst_gc_time_ovr','n_top10_ovr','n_win_ovr',
                         'best_pos_ovr_l3','avg_pos_ovr_l3','worst_pos_ovr_l3','best_time_ovr_l3','avg_time_ovr_l3','worst_time_ovr_l3',
                         'best_gc_ovr_l3','avg_gc_ovr_l3','worst_gc_ovr_l3','best_gc_time_ovr_l3','avg_gc_time_ovr_l3','worst_gc_time_ovr_l3',
                         'n_top10_l3','n_win_l3']

# Adding columns of Top 10 and winnig stage percetage
biker_by_stage_['perc_top10_ovr'] = biker_by_stage_['n_top10_ovr'] / biker_by_stage_['n_stages']
biker_by_stage_['perc_win_ovr'] = biker_by_stage_['n_win_ovr'] / biker_by_stage_['n_stages']
# biker_by_stage_

In [24]:
# Merging above results into DataFrame
# Each row is an information about each stage the biker have completed together with his personal info,
# counted statistics and historical data
biker_by_stage = pd.merge(stages, biker_by_stage_, left_on='name', right_on=biker_by_stage_.index)
biker_by_stage.head()

,name,biker_sys_nr,age,team,team_sys_nr,time_in_s,gc,gc_time_in_s,year,stage,...,best_gc_ovr_l3,avg_gc_ovr_l3,worst_gc_ovr_l3,best_gc_time_ovr_l3,avg_gc_time_ovr_l3,worst_gc_time_ovr_l3,n_top10_l3,n_win_l3,perc_top10_ovr,perc_win_ovr
0,Y.Hutarovich,100,31,AG2R La Mondiale,10072,20870.0,1,20860.0,2014,1,...,98,109.0,117,91441.0,104600.666667,112168.0,0.0,0.0,0.571429,0.142857
1,Y.Hutarovich,100,31,AG2R La Mondiale,10072,19455.0,2,40315.0,2014,2,...,98,109.0,117,91441.0,104600.666667,112168.0,0.0,0.0,0.571429,0.142857
2,Y.Hutarovich,100,31,AG2R La Mondiale,10072,13167.0,2,53482.0,2014,3,...,98,109.0,117,91441.0,104600.666667,112168.0,0.0,0.0,0.571429,0.142857
3,Y.Hutarovich,100,31,AG2R La Mondiale,10072,20609.0,3,74091.0,2014,4,...,98,109.0,117,91441.0,104600.666667,112168.0,0.0,0.0,0.571429,0.142857
4,Y.Hutarovich,100,31,AG2R La Mondiale,10072,17350.0,98,91441.0,2014,5,...,98,109.0,117,91441.0,104600.666667,112168.0,0.0,0.0,0.571429,0.142857


<br><br><br>
# Stages summary
Overall summary of all tours' stages
<br>

In [33]:
# Using 'url_tour_results' for creating stages summary data
url_stages_summary = []
for tour in url_tour_results:
    url_stages_summary.append(tour + '&k=stages')
url_stages_summary

['http://firstcycling.com/race.php?r=19&y=2014&k=stages',
 'http://firstcycling.com/race.php?r=19&y=2015&k=stages',
 'http://firstcycling.com/race.php?r=19&y=2016&k=stages',
 'http://firstcycling.com/race.php?r=19&y=2017&k=stages',
 'http://firstcycling.com/race.php?r=19&y=2018&k=stages']

In [34]:
# Web scraping stages summary data
stages_summary_ = []
for year in tour_years:
    for stage in [stage for stage in url_stages_summary if 'y='+str(year) in stage]:
        stages_summary__ = pd.read_html(stage)
        stages_summary__ = pd.DataFrame(stages_summary__[0])
        stages_summary__['year'] = year
        stages_summary_.append(stages_summary__)
stages_summary = pd.concat(stages_summary_, axis=0)
stages_summary.head(8)

,Unnamed: 0,Unnamed: 1,Date,Km,Unnamed: 4,Finish,Winner,Leader,Points,Mountains,Sprint,year
0,1,NaN,3.Aug,226.0,Results,Bygdoszcz,Y.Hutarovich,Y.Hutarovich,Y.Hutarovich,M.Paterski,J.Engoulvent,2014
1,2,NaN,4.Aug,226.0,Results,Warszawa,P.Vakoc,P.Vakoc,Y.Hutarovich,M.Paterski,P.Vakoc,2014
2,3,NaN,5.Aug,174.0,Results,Rzeszów,T.Bos,P.Vakoc,Y.Hutarovich,M.Paterski,P.Vakoc,2014
3,4,NaN,6.Aug,236.0,Results,Katowice,J.Van Genechten,P.Vakoc,Y.Hutarovich,M.Taciak,M.Krizek,2014
4,5,NaN,7.Aug,190.0,Results,Strbskie Pleso,R.Majka,P.Vakoc,Y.Hutarovich,M.Paterski,M.Krizek,2014
5,6,NaN,8.Aug,174.0,Results,Bukowina Tatrzanska,R.Majka,R.Majka,Y.Hutarovich,M.Paterski,M.Krizek,2014
6,7,NaN,9.Aug,25.0,Results,Krakow,K.Vandewalle,R.Majka,Y.Hutarovich,M.Paterski,M.Krizek,2014
0,1,NaN,2.Aug,122.0,Results,Warszawa,M.Kittel,M.Kittel,M.Kittel,A.Kurek,M.Mohoric,2015


### Specifying stage type by its webpage icon

In [35]:
# Since icons are named in Norwegian, I create Norwegian-English dictionary of stage types
# stage_names_nor = list(set(stage_type))
stage_names_nor = ['Flatt','Smaakupert','Smaakupert-MF','Fjell','Fjell-MF', 'Tempo']
stage_names_en = ['flat','hilly','hilly-mf','mountain','mountain-mf','itt']
stage_names = dict(zip(stage_names_nor, stage_names_en))
stage_type_recode = dict(zip(stage_names_nor, list(range(1,7)))) 

stage_names

{'Flatt': 'flat',
 'Smaakupert': 'hilly',
 'Smaakupert-MF': 'hilly-mf',
 'Fjell': 'mountain',
 'Fjell-MF': 'mountain-mf',
 'Tempo': 'itt'}

In [36]:
# Historically (but only before 2014 year), there were also two more stage types,
# but here I skip them as they are no longer valid.
# Skipped stage types: Bakketempo - mountain-itt; Ukjent - '?' (question mark); 

# Creating a list od all stage types for analyzed tours
stage_type = []
for stage in url_stages_summary:
    r = requests.get(stage)
    soup = BeautifulSoup(r.text,'html.parser')

    # Selecting all file names used in naming stage type
    stage_soup = soup.find_all('img')
    stage_img = re.findall('([A-Z][a-z].*?)\.gif', str(stage_soup))
    
    # Selecting only names correspoding to stage types 
    for name_ in stage_img:
        if name_ in stage_names_nor:
            stage_type.append(name_)
            
# stage_type

In [37]:
stages_summary.head()

,Unnamed: 0,Unnamed: 1,Date,Km,Unnamed: 4,Finish,Winner,Leader,Points,Mountains,Sprint,year
0,1,NaN,3.Aug,226.0,Results,Bygdoszcz,Y.Hutarovich,Y.Hutarovich,Y.Hutarovich,M.Paterski,J.Engoulvent,2014
1,2,NaN,4.Aug,226.0,Results,Warszawa,P.Vakoc,P.Vakoc,Y.Hutarovich,M.Paterski,P.Vakoc,2014
2,3,NaN,5.Aug,174.0,Results,Rzeszów,T.Bos,P.Vakoc,Y.Hutarovich,M.Paterski,P.Vakoc,2014
3,4,NaN,6.Aug,236.0,Results,Katowice,J.Van Genechten,P.Vakoc,Y.Hutarovich,M.Taciak,M.Krizek,2014
4,5,NaN,7.Aug,190.0,Results,Strbskie Pleso,R.Majka,P.Vakoc,Y.Hutarovich,M.Paterski,M.Krizek,2014


### Columns rearranging and cleaning

In [38]:
# Assigning stage type to appropriate column
stages_summary['Unnamed: 1'] = stage_type

# Recoding categorical stage type names to corresponding numerics and rearranging columns
stages_summary.columns = ['stage','stage_type','date','dist','to_drop','finish','stage_winner','stage_leader','points_leader','mountains_leader','sprint_leader','year']
stages_summary['stage_type_code'] = stages_summary['stage_type'].map(stage_type_recode)
stages_summary = stages_summary[['date','year','stage','stage_type','stage_type_code','dist','to_drop','finish','stage_winner','stage_leader','points_leader','mountains_leader','sprint_leader']]

# Mapping English names to Norwegian descriptions
stages_summary['stage_type'] = stages_summary['stage_type'].map(stage_names)

stages_summary.drop('to_drop', axis=1, inplace=True)

# Selecting only appropriate columns
stages_summary = stages_summary.loc[:, 'date':'finish'].reset_index(drop=True)
stages_summary.head()

,date,year,stage,stage_type,stage_type_code,dist,finish
0,3.Aug,2014,1,flat,1,226.0,Bygdoszcz
1,4.Aug,2014,2,flat,1,226.0,Warszawa
2,5.Aug,2014,3,flat,1,174.0,Rzeszów
3,6.Aug,2014,4,flat,1,236.0,Katowice
4,7.Aug,2014,5,hilly,2,190.0,Strbskie Pleso


### Parsing stage date into appropriate format

In [39]:
from datetime import timedelta

In [40]:
stages_summary['date'] = stages_summary['date'] + '.' + stages_summary['year'].astype('str')

In [41]:
stages_summary['date'] = pd.to_datetime(stages_summary['date'], format='%d.%b.%Y')
stages_summary['date'][:3]

0   2014-08-03
1   2014-08-04
2   2014-08-05
Name: date, dtype: datetime64[ns]

### Weather info

In [42]:
# Usefull information that can have an impact on the stage results are:
# Precipitation(0/1; TBD), Temp(float), Cloud overcast and a few others
# Data scraped from: https://dane.imgw.pl/data/dane_pomiarowo_obserwacyjne/

'''
Technical names for different weather parameters with numbers indicating lenght of the value:

Station code                         9
Station name                        30
Year                                 4
Month                                2
Day                                  2
Hour                                 2
Air temperature                    6/1
TEMP measurement status              1
Wetted thermometer temperature     6/1
TTZW measurement status              1
Ice indicator                        1
Ventilation indicator                1
Relative humidity                    5
WLGW measurement indicator           1
Wind direction code                  3
DKDK measurement indicator           1
Wind speed                           5
FWR measurement indicator            1
Overall overcast                     5
ZOGK measurement indicator           1
Visibility                           5
WID measurement indicator            1

Status "8" means lack of measurement
Overcast:
        - 0 - 10 scale up to 31.12.1988
        - 0 – 8 scale up to 01.01.1989
'''

names = ['station_code','station_name','year','month','day','hour',
         'temp','temp_status','wet_temp','ttzw_status','ice_ind','vent_ind',
         'rel_humid','wlgw_status','wind_dir','dkdk_status','wind_speed','fwr_status',
         'clouds_ovrl','zogk_status','visibility','wid_status']

weather_08_14 = pd.read_csv('data/weather_k_t_08_2014.csv', encoding = 'ISO-8859-2', header=None, names=names)
weather_08_15 = pd.read_csv('data/weather_k_t_08_2015.csv', encoding = 'ISO-8859-2', header=None, names=names)
weather_07_16 = pd.read_csv('data/weather_k_t_07_2016.csv', encoding = 'ISO-8859-2', header=None, names=names)
weather_07_17 = pd.read_csv('data/weather_k_t_07_2017.csv', encoding = 'ISO-8859-2', header=None, names=names)
weather_08_17 = pd.read_csv('data/weather_k_t_08_2017.csv', encoding = 'ISO-8859-2', header=None, names=names)
weather_08_18 = pd.read_csv('data/weather_k_t_08_2018.csv', encoding = 'ISO-8859-2', header=None, names=names)

In [43]:
# Gathering all above weather information into one DataFrame
weather_info = pd.concat([weather_08_14, weather_08_15,weather_07_16,
                          weather_07_17, weather_08_17, weather_08_18], axis=0)
# Choosing time of measurement as 12 o'clock
weather_info = weather_info[weather_info.hour == 12].reset_index(drop=True)
weather_info.head()

,station_code,station_name,year,month,day,hour,temp,temp_status,wet_temp,ttzw_status,...,rel_humid,wlgw_status,wind_dir,dkdk_status,wind_speed,fwr_status,clouds_ovrl,zogk_status,visibility,wid_status
0,249180010,PSZCZYNA,2014,8,1,12,22.4,NaN,0.0,8.0,...,0,8.0,C,NaN,0,NaN,5,NaN,6,NaN
1,249180010,PSZCZYNA,2014,8,2,12,29.5,NaN,0.0,8.0,...,0,8.0,E,NaN,2,NaN,0,NaN,8,NaN
2,249180010,PSZCZYNA,2014,8,3,12,29.4,NaN,0.0,8.0,...,0,8.0,SSE,NaN,2,NaN,1,NaN,8,NaN
3,249180010,PSZCZYNA,2014,8,4,12,20.2,NaN,0.0,8.0,...,0,8.0,WSW,NaN,2,NaN,4,NaN,8,NaN
4,249180010,PSZCZYNA,2014,8,5,12,22.4,NaN,0.0,8.0,...,0,8.0,SE,NaN,1,NaN,3,NaN,6,NaN


In [44]:
# Merging all weather date columns to compare them later with stage dates
weather_info['date'] = weather_info['year'].astype('str') + '.' +\
                       weather_info['month'].astype('str') + '.' +\
                       weather_info['day'].astype('str')
weather_info['date'] = pd.to_datetime(weather_info['date'], format='%Y.%m.%d')
weather_info.head()

,station_code,station_name,year,month,day,hour,temp,temp_status,wet_temp,ttzw_status,...,wlgw_status,wind_dir,dkdk_status,wind_speed,fwr_status,clouds_ovrl,zogk_status,visibility,wid_status,date
0,249180010,PSZCZYNA,2014,8,1,12,22.4,NaN,0.0,8.0,...,8.0,C,NaN,0,NaN,5,NaN,6,NaN,2014-08-01
1,249180010,PSZCZYNA,2014,8,2,12,29.5,NaN,0.0,8.0,...,8.0,E,NaN,2,NaN,0,NaN,8,NaN,2014-08-02
2,249180010,PSZCZYNA,2014,8,3,12,29.4,NaN,0.0,8.0,...,8.0,SSE,NaN,2,NaN,1,NaN,8,NaN,2014-08-03
3,249180010,PSZCZYNA,2014,8,4,12,20.2,NaN,0.0,8.0,...,8.0,WSW,NaN,2,NaN,4,NaN,8,NaN,2014-08-04
4,249180010,PSZCZYNA,2014,8,5,12,22.4,NaN,0.0,8.0,...,8.0,SE,NaN,1,NaN,3,NaN,6,NaN,2014-08-05


In [45]:
# Based on known stage location I manually chose corresponding meteo stations 
# This is to be further tweaked for choosing more accurate (closer) meteo st. locations

weather_stages = pd.DataFrame()
for year in tour_years:
    # Stage 1
    st1_weather = weather_info[weather_info.date.isin(stages_summary.date) \
                    & (weather_info.year == year) \
                    & weather_info.station_name.str.contains('KRAK')].iloc[0] \
                    [['temp','rel_humid','wind_dir','wind_speed','clouds_ovrl','visibility']]
    # Stage 2
    st2_weather = weather_info[weather_info.date.isin(stages_summary.date) \
                    & (weather_info.year == year) \
                    & weather_info.station_name.str.contains('DRONI')].iloc[1] \
                    [['temp','rel_humid','wind_dir','wind_speed','clouds_ovrl','visibility']]
    # Stage 3
    st3_weather = weather_info[weather_info.date.isin(stages_summary.date) \
                    & (weather_info.year == year) \
                    & weather_info.station_name.str.contains('BRENNA')].iloc[2] \
                    [['temp','rel_humid','wind_dir','wind_speed','clouds_ovrl','visibility']]
    # Stage 4
    st4_weather = weather_info[weather_info.date.isin(stages_summary.date) \
                    & (weather_info.year == year) \
                    & weather_info.station_name.str.contains('DRONI')].iloc[3] \
                    [['temp','rel_humid','wind_dir','wind_speed','clouds_ovrl','visibility']]
    # Stage 5
    st5_weather = weather_info[weather_info.date.isin(stages_summary.date) \
                    & (weather_info.year == year) \
                    & weather_info.station_name.str.contains('CHORZ')].iloc[4] \
                    [['temp','rel_humid','wind_dir','wind_speed','clouds_ovrl','visibility']]
    # Stage 6
    st6_weather = weather_info[weather_info.date.isin(stages_summary.date) \
                    & (weather_info.year == year) \
                    & weather_info.station_name.str.contains('LIMA')].iloc[5] \
                    [['temp','rel_humid','wind_dir','wind_speed','clouds_ovrl','visibility']]
    # Stage 7
    st7_weather = weather_info[weather_info.date.isin(stages_summary.date) \
                    & (weather_info.year == year) \
                    & weather_info.station_name.str.contains('BUKOW')].iloc[6] \
                    [['temp','rel_humid','wind_dir','wind_speed','clouds_ovrl','visibility']]
    weather_stages = weather_stages.append([st1_weather, st2_weather, st3_weather, st4_weather,
                           st5_weather, st6_weather, st7_weather]).reset_index(drop=True)

In [46]:
# Removing white space from values
weather_stages.wind_dir = weather_stages.wind_dir.str.strip()

In [47]:
# Chosen weather info for all tour stages summaries
weather_stages.head(8)

,temp,rel_humid,wind_dir,wind_speed,clouds_ovrl,visibility
0,31.5,38,ESE,2,6,9
1,26.6,61,NW,4,5,7
2,22.5,65,SSE,1,4,6
3,22.7,52,N,3,4,8
4,25.5,66,ENE,2,1,8
5,21.4,73,NW,4,7,8
6,22.9,0,NaN,0,5,7
7,27.1,38,SSE,1,7,9


### Joining stages summary and weather info dataframes. The output is `stages_summary`

In [48]:
stages_summary = pd.concat([stages_summary, weather_stages], axis=1)
# Removing stage without results (6th stage of 2016)
stages_summary = stages_summary.drop(19, axis=0).reset_index(drop=True)

### Bikers finishing positions in all stages

In [49]:
position_list_ = []
for k in biker_dict_stage.keys():
    position_list_.append(stages[stages['name'] == k][['year','stage','position']])

position = position_list_[0]
for i in range(len(position_list_)):
    position = position.merge(position_list_[i], on=['year','stage'], how='outer')

position = position.sort_values(['year','stage'], ascending=True).reset_index(drop=True)
position = position.iloc[:,3:]
position.columns = [k for k in biker_dict_stage.keys()]
stage_positions = pd.concat([stages_summary.loc[:,'date':'finish'], position], axis=1)

In [50]:
stage_positions.head()

,date,year,stage,stage_type,stage_type_code,dist,finish,Y.Hutarovich,R.Maikin,M.Mori,...,L.Boom,M.Schwarzmann,L.Hofstede,B.Eisel,L.Owen,A.Frame,P.Bauhaus,D.McLay,J.Cardona,K.Magnusson
0,2014-08-03,2014,1,flat,1,226.0,Bygdoszcz,1.0,2.0,3.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,2014-08-04,2014,2,flat,1,226.0,Warszawa,5.0,11.0,13.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,2014-08-05,2014,3,flat,1,174.0,Rzeszów,7.0,36.0,54.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,2014-08-06,2014,4,flat,1,236.0,Katowice,5.0,13.0,44.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,2014-08-07,2014,5,hilly,2,190.0,Strbskie Pleso,131.0,999.0,57.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


<br><br><br>
# Bikers statistics and overall description scraping
<br>

In [51]:
# Overall bikers list
biker_base_url = 'http://firstcycling.com/rider.php?r='

bikers_ranking_urls = []
for k,v in biker_dict_stage.items():
    bikers_ranking_urls.append(biker_base_url + v)
print('Number of all bikers:', len(bikers_ranking_urls))
# pd.DataFrame(bikers_ranking_urls)[0]

Number of all bikers: 513


### Scraping bikers stats

**(NOTICE: This operation takes around 20 minutes)**

In [40]:
# STOP

In [238]:
for i,c in enumerate(bikers_ranking_urls):
    if i == 82:
        print(i,c)

82 http://firstcycling.com/rider.php?r=83


In [239]:
tour_year_start = 2014
tour_year_stop = 2018

In [ ]:
start_time = time.clock()
bikers_stats_ = pd.DataFrame()
i = -1
for url_ in bikers_ranking_urls:

    # Stats scraping with the season-by-season separation
    r = requests.get(url_)
    table_ = pd.read_html(r.text)   
    
    # Skipping data when year 2014 non-existent
    # Settig 'Year' column as an index for dataframe transposition
    try:
        biker_rank = table_[0].set_index('Year').loc[:str(tour_year_start),:]
    except (ValueError, KeyError):
        biker_rank = table_[0].set_index('Year').loc[:str(tour_year_start + 1),:]
        
    biker_rank.columns = ['div','team','fc_rank','to_drop','to_drop',
                          'uci_rank','to_drop','race_days','wins','year_km','to_drop']
    biker_rank.drop(['to_drop'], axis=1, inplace=True)
    # Dataframe transposition
    biker_rank = biker_rank.stack().to_frame().T
    # Renaming stacked column names
    biker_rank.columns = ['{}_{}'.format(*c) for c in biker_rank.columns]  
    # Removing duplicate rows
    biker_rank = biker_rank.loc[:,~biker_rank.columns.duplicated()]
    
    # Physical stats, nationality, won jerseys, all-time victories and followers number extraction
    soup = BeautifulSoup(r.text,'html.parser')
    stats = soup.find_all('div', class_='back')
    
    # Not all bikers have all data available
    nation = re.findall('nat=.*?>(\w.*?)<|$', str(soup))[0]
    try:
        height = re.findall('Height.*?(\d{1}\.\d{2})|$', str(soup))[0] 
    except (ValueError, IndexError):
        height = np.NaN
    try:
        wins_search = re.search('Victories.*?<h2', str(stats)).group() 
        total_wins = len(re.findall('\d{4}(,|\))', wins_search))
    except (AttributeError, ValueError, IndexError):
        total_wins = 0
    try:
        youth_search = re.search('Youth.*?<\/p', str(stats)).group()
        youth_jersey = len(re.findall('[( ,](\d{4})', youth_search))
    except (AttributeError, ValueError, IndexError):
        youth_jersey = 0
    try:
        point_search = re.search('Points.*?<\/p', str(stats)).group()
        point_jersey = len(re.findall('[( ,](\d{4})', point_search))
    except (AttributeError, ValueError, IndexError):
        point_jersey = 0
    try:
        mount_search = re.search('Mount.*?<\/p', str(stats)).group()
        mount_jersey = len(re.findall('[( ,](\d{4})', mount_search))
    except (AttributeError, ValueError, IndexError):
        mount_jersey = 0
    try:
        sprint_search = re.search('Sprint.*?<\/p', str(stats)).group()
        sprint_jersey = len(re.findall('[( ,](\d{4})', sprint_search))
    except (AttributeError, ValueError, IndexError):
        sprint_jersey = 0
    try:
        followers_soup = soup.find_all('p', class_='small')
        followers = len(re.findall('ID', str(followers_soup)))
    except (AttributeError, ValueError, IndexError):
        followers = 0
    
    # All separate information merged into one 'stats_table' dataframe
    results_dict = {
        'nation':nation,
        'height':height,
        'total_wins':total_wins,
        'youth_jerseys':youth_jersey,
        'point_jerseys':point_jersey,
        'mount_jerseys':mount_jersey,
        'sprint_jerseys':sprint_jersey,
        'followers':followers
    }
    stats_table = pd.DataFrame(data=results_dict, index=[0])
    
    # Merging dataframes
    biker_rank = pd.concat([biker_rank, stats_table], axis=1)
    i += 1
    print('Scraped biker no.', i, '/', len(bikers_ranking_urls),nation, height, total_wins, youth_jersey, point_jersey,
          mount_jersey, sprint_jersey, followers)
    bikers_stats_ = bikers_stats_.append(biker_rank, sort=False)

print('-'*50)
print('Data gathered in: {} minutes'.format((time.clock()-start_time)/60))

In [242]:
# Alphabetically column sorting
bikers_stats_.sort_index(axis=1, inplace=True)

# Assigning biker names from dictionary to the scraped data
bikers_stats_['name'] = [k for k,v in biker_dict_stage.items()][:len(bikers_stats_)]

# Resetting index for further dataframe concatenation
bikers_stats_.reset_index(drop=True, inplace=True)

# Replacing missed height values
bikers_stats_['height'].replace('', np.NaN, inplace=True)

### Merging data into `bikers_stats`

**Choosing only years of 2015 - 2018 due to much missing data in earlier years**

In [243]:
# From the whole scraped historical results dataset (2013-2019) choosing only years 2015-2018 due to many missing values before that timepoint
bikers_stats = pd.concat([bikers_stats_.loc[:,'2015_div':'2018_year_km'], bikers_stats_.iloc[:,-9:]], axis=1)
bikers_stats.head(3)

,2015_div,2015_fc_rank,2015_race_days,2015_team,2015_uci_rank,2015_wins,2015_year_km,2016_div,2016_fc_rank,2016_race_days,...,2018_year_km,followers,height,mount_jerseys,nation,point_jerseys,sprint_jerseys,total_wins,youth_jerseys,name
0,PROF,115,70,Bretagne - Séché Environnement,-,3,11430.9,PROF,430,59,...,NaN,0,1.79,0,Belarus,6,0,8,0,Y.Hutarovich
1,PROF,301,54,RusVelo,-,1,8473.1,PROF,161,63,...,7737.0,0,NaN,0,Russia,2,0,0,0,R.Maikin
2,PRT,640,73,Lampre - Merida,-,-,11960.4,PRT,375,70,...,10709.4,0,1.72,0,Italy,0,0,4,0,M.Mori


### Setting final biker class dataframe. The output is `biker_class`

In [244]:
biker_by_stage.columns

Index(['name', 'biker_sys_nr', 'age', 'team', 'team_sys_nr', 'time_in_s', 'gc',
       'gc_time_in_s', 'year', 'stage', 'position', 'n_tours', 'n_stages',
       'avg_year', 'best_pos_ovr', 'avg_pos_ovr', 'worst_pos_ovr',
       'best_time_ovr', 'avg_time_ovr', 'worst_time_ovr', 'best_gc_ovr',
       'avg_gc_ovr', 'worst_gc_ovr', 'best_gc_time_ovr', 'avg_gc_time_ovr',
       'worst_gc_time_ovr', 'n_top10_ovr', 'n_win_ovr', 'best_pos_ovr_l3',
       'avg_pos_ovr_l3', 'worst_pos_ovr_l3', 'best_time_ovr_l3',
       'avg_time_ovr_l3', 'worst_time_ovr_l3', 'best_gc_ovr_l3',
       'avg_gc_ovr_l3', 'worst_gc_ovr_l3', 'best_gc_time_ovr_l3',
       'avg_gc_time_ovr_l3', 'worst_gc_time_ovr_l3', 'n_top10_l3', 'n_win_l3',
       'perc_top10_ovr', 'perc_win_ovr'],
      dtype='object')

In [245]:
bikers_stats.columns

Index(['2015_div', '2015_fc_rank', '2015_race_days', '2015_team',
       '2015_uci_rank', '2015_wins', '2015_year_km', '2016_div',
       '2016_fc_rank', '2016_race_days', '2016_team', '2016_uci_rank',
       '2016_wins', '2016_year_km', '2017_div', '2017_fc_rank',
       '2017_race_days', '2017_team', '2017_uci_rank', '2017_wins',
       '2017_year_km', '2018_div', '2018_fc_rank', '2018_race_days',
       '2018_team', '2018_uci_rank', '2018_wins', '2018_year_km', 'followers',
       'height', 'mount_jerseys', 'nation', 'point_jerseys', 'sprint_jerseys',
       'total_wins', 'youth_jerseys', 'name'],
      dtype='object')

In [246]:
# Creating biker class dataset using all-stage biker data and counted statistics
biker_class = pd.merge(biker_by_stage, bikers_stats, left_on='name', right_on='name')
biker_class.head()

,name,biker_sys_nr,age,team,team_sys_nr,time_in_s,gc,gc_time_in_s,year,stage,...,2018_wins,2018_year_km,followers,height,mount_jerseys,nation,point_jerseys,sprint_jerseys,total_wins,youth_jerseys
0,Y.Hutarovich,100,31,AG2R La Mondiale,10072,20870.0,1,20860.0,2014,1,...,NaN,NaN,0,1.79,0,Belarus,6,0,8,0
1,Y.Hutarovich,100,31,AG2R La Mondiale,10072,19455.0,2,40315.0,2014,2,...,NaN,NaN,0,1.79,0,Belarus,6,0,8,0
2,Y.Hutarovich,100,31,AG2R La Mondiale,10072,13167.0,2,53482.0,2014,3,...,NaN,NaN,0,1.79,0,Belarus,6,0,8,0
3,Y.Hutarovich,100,31,AG2R La Mondiale,10072,20609.0,3,74091.0,2014,4,...,NaN,NaN,0,1.79,0,Belarus,6,0,8,0
4,Y.Hutarovich,100,31,AG2R La Mondiale,10072,17350.0,98,91441.0,2014,5,...,NaN,NaN,0,1.79,0,Belarus,6,0,8,0


In [247]:
# Rearranging columns for clearness
cols = biker_class.columns.tolist()
cols = cols[-8:] + cols[:-8]
biker_class = biker_class[cols]
biker_class = biker_class.reindex(columns=[
    'name', 'nation','age','height','team','year','stage','position','time_in_s','gc','gc_time_in_s',
    'mount_jerseys','point_jerseys','sprint_jerseys','youth_jerseys','followers','total_wins',
    'biker_sys_nr','team_sys_nr']+cols[19:])       

### Adding `stages_summary` information

In [248]:
biker_class = biker_class.merge(stages_summary,on=['year','stage'], how='inner')

### Defining `y_label` (dependend variable)

In [249]:
# Defining the label (dependent variable)
# 1 means being in Top 10 position at the given stage, 0 means being out of Top 10
biker_class['y_label'] = np.where(biker_class['position'] <= 10, 1, 0)

<br><br><br>
# Saving dataframes

### Overall stages summary: `stages_summary`

In [250]:
stages_summary.to_csv('preprocessed/pre_stages_summary.csv', index=False)
stages_summary.head()

,date,year,stage,stage_type,stage_type_code,dist,finish,temp,rel_humid,wind_dir,wind_speed,clouds_ovrl,visibility
0,2014-08-03,2014,1,flat,1,226.0,Bygdoszcz,31.5,38,ESE,2,6,9
1,2014-08-04,2014,2,flat,1,226.0,Warszawa,26.6,61,NW,4,5,7
2,2014-08-05,2014,3,flat,1,174.0,Rzeszów,22.5,65,SSE,1,4,6
3,2014-08-06,2014,4,flat,1,236.0,Katowice,22.7,52,N,3,4,8
4,2014-08-07,2014,5,hilly,2,190.0,Strbskie Pleso,25.5,66,ENE,2,1,8


### Bikers finishing positions through stages: `stage_positions`

In [251]:
stage_positions.to_csv('preprocessed/pre_stage_positions.csv', index=False)
stage_positions.head()

,date,year,stage,stage_type,stage_type_code,dist,finish,Y.Hutarovich,R.Maikin,M.Mori,...,L.Boom,M.Schwarzmann,L.Hofstede,B.Eisel,L.Owen,A.Frame,P.Bauhaus,D.McLay,J.Cardona,K.Magnusson
0,2014-08-03,2014,1,flat,1,226.0,Bygdoszcz,1.0,2.0,3.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,2014-08-04,2014,2,flat,1,226.0,Warszawa,5.0,11.0,13.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,2014-08-05,2014,3,flat,1,174.0,Rzeszów,7.0,36.0,54.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,2014-08-06,2014,4,flat,1,236.0,Katowice,5.0,13.0,44.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,2014-08-07,2014,5,hilly,2,190.0,Strbskie Pleso,131.0,999.0,57.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


### Tour results by stages: `stages`

In [252]:
stages.to_csv('preprocessed/pre_stages.csv', index=False)
stages.head(2)

,name,biker_sys_nr,age,team,team_sys_nr,time_in_s,gc,gc_time_in_s,year,stage,position
0,Y.Hutarovich,100,31,AG2R La Mondiale,10072,20870.0,1,20860.0,2014,1,1
1,R.Maikin,14004,24,RusVelo,3482,20870.0,2,20864.0,2014,1,2


### Biker by stage results: `biker_by_stage`

In [253]:
biker_by_stage.to_csv('preprocessed/pre_biker_by_stage.csv')
biker_by_stage.head(2)

,name,biker_sys_nr,age,team,team_sys_nr,time_in_s,gc,gc_time_in_s,year,stage,...,best_gc_ovr_l3,avg_gc_ovr_l3,worst_gc_ovr_l3,best_gc_time_ovr_l3,avg_gc_time_ovr_l3,worst_gc_time_ovr_l3,n_top10_l3,n_win_l3,perc_top10_ovr,perc_win_ovr
0,Y.Hutarovich,100,31,AG2R La Mondiale,10072,20870.0,1,20860.0,2014,1,...,98,109.0,117,91441.0,104600.666667,112168.0,0.0,0.0,0.571429,0.142857
1,Y.Hutarovich,100,31,AG2R La Mondiale,10072,19455.0,2,40315.0,2014,2,...,98,109.0,117,91441.0,104600.666667,112168.0,0.0,0.0,0.571429,0.142857


### Historical bikers data: `biker_stats`

In [254]:
bikers_stats.to_csv('preprocessed/pre_biker_stats.csv')
bikers_stats.head(2)

,2015_div,2015_fc_rank,2015_race_days,2015_team,2015_uci_rank,2015_wins,2015_year_km,2016_div,2016_fc_rank,2016_race_days,...,2018_year_km,followers,height,mount_jerseys,nation,point_jerseys,sprint_jerseys,total_wins,youth_jerseys,name
0,PROF,115,70,Bretagne - Séché Environnement,-,3,11430.9,PROF,430,59,...,NaN,0,1.79,0,Belarus,6,0,8,0,Y.Hutarovich
1,PROF,301,54,RusVelo,-,1,8473.1,PROF,161,63,...,7737.0,0,NaN,0,Russia,2,0,0,0,R.Maikin


### Final biker class dataframe: `biker_class`

In [255]:
biker_class.to_csv('preprocessed/pre_biker_class.csv')
biker_class.head(2)

,name,nation,age,height,team,year,stage,position,time_in_s,gc,...,stage_type_code,dist,finish,temp,rel_humid,wind_dir,wind_speed,clouds_ovrl,visibility,y_label
0,Y.Hutarovich,Belarus,31,1.79,AG2R La Mondiale,2014,1,1,20870.0,1,...,1,226.0,Bygdoszcz,31.5,38,ESE,2,6,9,1
1,R.Maikin,Russia,24,NaN,RusVelo,2014,1,2,20870.0,2,...,1,226.0,Bygdoszcz,31.5,38,ESE,2,6,9,1


<br><br><br>
# Cleaning data

**Next notebook:** [Part 2: Data statistics and visualization](tdp_2_3_data_statistics_visualization.ipynb)

In [256]:
biker_class = pd.read_csv('preprocessed/pre_biker_class.csv', index_col=0)
biker_class.shape

(5255, 92)

## Removing columns with many missing values or too many levels for the one-hot-encoding

In [257]:
# Removing columns with many missing values and information not needed
biker_class.drop(['height','biker_sys_nr','team_sys_nr','stage_type'], axis=1, inplace=True)
biker_class.shape

(5255, 88)

## Fixing data types & further low-information columns removal

In [258]:
# biker_class.dtypes

In [259]:
# Fixing date column
biker_class['date'] = pd.to_datetime(biker_class['date'])

In [260]:
# Wrong-type columns
biker_class.loc[:,'2015_div':'2018_year_km'].dtypes

2015_div          object
2015_fc_rank      object
2015_race_days    object
2015_team         object
2015_uci_rank     object
2015_wins         object
2015_year_km      object
2016_div          object
2016_fc_rank      object
2016_race_days    object
2016_team         object
2016_uci_rank     object
2016_wins         object
2016_year_km      object
2017_div          object
2017_fc_rank      object
2017_race_days    object
2017_team         object
2017_uci_rank     object
2017_wins         object
2017_year_km      object
2018_div          object
2018_fc_rank      object
2018_race_days    object
2018_team         object
2018_uci_rank     object
2018_wins         object
2018_year_km      object
dtype: object

In [261]:
# Looking closer at the content we see there are many values as '-'
biker_class.loc[:,'2015_div':'2018_year_km'].head()

,2015_div,2015_fc_rank,2015_race_days,2015_team,2015_uci_rank,2015_wins,2015_year_km,2016_div,2016_fc_rank,2016_race_days,...,2017_uci_rank,2017_wins,2017_year_km,2018_div,2018_fc_rank,2018_race_days,2018_team,2018_uci_rank,2018_wins,2018_year_km
0,PROF,115,70,Bretagne - Séché Environnement,-,3,11430.9,PROF,430,59,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,PROF,301,54,RusVelo,-,1,8473.1,PROF,161,63,...,-,-,8936.9,PROF,807,46,Gazprom - RusVelo,-,-,7737.0
2,PRT,640,73,Lampre - Merida,-,-,11960.4,PRT,375,70,...,-,-,8602.1,PRT,1263,67,UAE Team Emirates,-,-,10709.4
3,CONT,460,62,Optum p/b Kelly Benefit Strategies,-,1,8655.5,CONT,619,40,...,-,1,9546.0,PROF,196,78,Israel Cycling Academy,-,1,12967.3
4,PRT,360,72,Team Katusha,-,1,11969.9,PRT,1340,85,...,-,-,13660.8,PRT,1097,23,Team Katusha - Alpecin,-,-,3644.7


In [262]:
# Replacing '-' with NaNs
biker_class.loc[:,'2015_div':'2018_year_km'] = biker_class.loc[:,'2015_div':'2018_year_km'].replace('-', np.NaN)

In [263]:
# Changing type of 'object' columns to their proper data type
rank_ix = biker_class.columns[biker_class.columns.str.contains('_rank')]
biker_class[rank_ix] = biker_class[rank_ix].astype('float')
biker_class[rank_ix].dtypes

2015_fc_rank     float64
2015_uci_rank    float64
2016_fc_rank     float64
2016_uci_rank    float64
2017_fc_rank     float64
2017_uci_rank    float64
2018_fc_rank     float64
2018_uci_rank    float64
dtype: object

In [264]:
# Checking null-values in selected columns
biker_class[rank_ix].isnull().sum()

2015_fc_rank      147
2015_uci_rank    5255
2016_fc_rank      256
2016_uci_rank    5255
2017_fc_rank      465
2017_uci_rank    5255
2018_fc_rank      793
2018_uci_rank    5255
dtype: int64

In [265]:
# Further column removal due to many missing values
biker_class.drop(list(biker_class.filter(regex='_uci_rank')), axis=1, inplace=True)
biker_class.shape

(5255, 84)

## Fixing unique names

In [266]:
biker_class['finish'].unique()

array(['Bygdoszcz', 'Warszawa', 'Rzeszów', 'Katowice', 'Strbskie Pleso',
       'Bukowina Tatrzanska', 'Krakow', 'Warsaw', 'Nowy Sacz', 'Zakopane',
       'Kraków', 'Szczyrk', 'Zabrze', 'Bukowina Tatrzańska',
       'Dąbrowa Górnicza', 'Nowy Sącz', 'Bielsko-Biała',
       'Bukovina Resort'], dtype=object)

In [267]:
biker_class['finish'].replace({'Warsaw':'Warszawa','Bukowina Tatrzanska':'Bukowina Tatrzańska',
                               'Bukovina Resort':'Bukowina Tatrzańska','Krakow':'Kraków',
                               'Nowy Sacz':'Nowy Sącz','Bygdoszcz':'Bydgoszcz'},
                              inplace=True)

In [268]:
biker_class['finish'].unique()

array(['Bydgoszcz', 'Warszawa', 'Rzeszów', 'Katowice', 'Strbskie Pleso',
       'Bukowina Tatrzańska', 'Kraków', 'Nowy Sącz', 'Zakopane',
       'Szczyrk', 'Zabrze', 'Dąbrowa Górnicza', 'Bielsko-Biała'],
      dtype=object)

## Imputing data

In [269]:
# Checking missing values
biker_class_null_cols = biker_class.columns[biker_class.isnull().any()]
biker_class[biker_class_null_cols].isnull().sum()

2015_div           254
2015_fc_rank       147
2015_race_days      88
2015_team           67
2015_wins         3123
2015_year_km        88
2016_div           200
2016_fc_rank       256
2016_race_days     114
2016_team           88
2016_wins         3623
2016_year_km       114
2017_div           385
2017_fc_rank       465
2017_race_days     329
2017_team          302
2017_wins         3728
2017_year_km       329
2018_div           666
2018_fc_rank       793
2018_race_days     674
2018_team          541
2018_wins         3753
2018_year_km       674
wind_dir           291
dtype: int64

### Categorical data

In [270]:
biker_class.select_dtypes('object').isnull().sum()

name                 0
nation               0
team                 0
2015_div           254
2015_race_days      88
2015_team           67
2015_wins         3123
2015_year_km        88
2016_div           200
2016_race_days     114
2016_team           88
2016_wins         3623
2016_year_km       114
2017_div           385
2017_race_days     329
2017_team          302
2017_wins         3728
2017_year_km       329
2018_div           666
2018_race_days     674
2018_team          541
2018_wins         3753
2018_year_km       674
finish               0
wind_dir           291
dtype: int64

In [271]:
# Imputing missing categorical weather data with the most frequent value
biker_class['wind_dir'].fillna(biker_class['wind_dir'].value_counts().index[0], inplace=True)

In [272]:
# Imputing missing values in '_div' and '_team' data as new category 'NONE'
biker_class.loc[:,'2015_div':'2018_year_km':3] = biker_class.loc[:,'2015_div':'2018_year_km':3].fillna('NONE')

### Numeric data

In [273]:
# Missing values
biker_class_null_cols = biker_class.columns[biker_class.isnull().any()]
biker_class[biker_class_null_cols].isnull().sum()

2015_fc_rank       147
2015_race_days      88
2015_wins         3123
2015_year_km        88
2016_fc_rank       256
2016_race_days     114
2016_wins         3623
2016_year_km       114
2017_fc_rank       465
2017_race_days     329
2017_wins         3728
2017_year_km       329
2018_fc_rank       793
2018_race_days     674
2018_wins         3753
2018_year_km       674
dtype: int64

In [274]:
# Filling missing data with high number to mark them as very low in a ranking
rank_ix = biker_class.columns[biker_class.columns.str.contains('_fc_rank')]
biker_class[rank_ix] = biker_class[rank_ix].fillna('999999')
biker_class[rank_ix].head()

,2015_fc_rank,2016_fc_rank,2017_fc_rank,2018_fc_rank
0,115,430,999999,999999
1,301,161,428,807
2,640,375,1257,1263
3,460,619,276,196
4,360,1340,765,1097


In [275]:
# Imputing 0s for the rest of value-missing columns as the Biker was not riding at that period
biker_class_null_cols = biker_class.columns[biker_class.isnull().any()]
biker_class[biker_class_null_cols] = biker_class[biker_class_null_cols].fillna(0)
biker_class[biker_class_null_cols].head()

,2015_race_days,2015_wins,2015_year_km,2016_race_days,2016_wins,2016_year_km,2017_race_days,2017_wins,2017_year_km,2018_race_days,2018_wins,2018_year_km
0,70,3,11430.9,59,1,9262.8,0,0,0,0,0,0
1,54,1,8473.1,63,2,10287.0,52,0,8936.9,46,0,7737.0
2,73,0,11960.4,70,0,11537.7,55,0,8602.1,67,0,10709.4
3,62,1,8655.5,40,1,5630.7,61,1,9546.0,78,1,12967.3
4,72,1,11969.9,85,0,14288.5,85,0,13660.8,23,0,3644.7


### Sorting by 'date' column to avoid data leakage later during splitting to train/test batches in modeling part

In [276]:
biker_class.sort_values('date', ascending=True, inplace=True)
biker_class.reset_index(drop=True, inplace=True)

## Saving final cleaned dataframe: `biker_class`

In [277]:
biker_class.to_csv('processed/clean_biker_class.csv')
print(biker_class.shape)
biker_class.head(2)

(5255, 84)


,name,nation,age,team,year,stage,position,time_in_s,gc,gc_time_in_s,...,stage_type_code,dist,finish,temp,rel_humid,wind_dir,wind_speed,clouds_ovrl,visibility,y_label
0,Y.Hutarovich,Belarus,31,AG2R La Mondiale,2014,1,1,20870.0,1,20860.0,...,1,226.0,Bydgoszcz,31.5,38,ESE,2,6,9,1
1,T.De Gendt,Belgium,28,Omega Pharma-Quick Step,2014,1,108,20992.0,109,20992.0,...,1,226.0,Bydgoszcz,31.5,38,ESE,2,6,9,0


--------------

**Next notebook:** [Part 2: Data statistics and visualization](tdp_2_3_data_statistics_visualization.ipynb)